In [1]:
import numpy as np
import scvelo as scv
import scanpy as sc
import sys
import torch
import os.path
import pandas as pd
import anndata as ad
import os
import velovae as vv
from scipy.sparse import csr_matrix

SEED = 2024
np.random.seed(SEED)


In [2]:
adata = sc.read_h5ad("LSK_lineage.h5ad")
print(adata)
adata.layers['spliced'] = csr_matrix(adata.layers['spliced'])
adata.layers['unspliced'] = csr_matrix(adata.layers['unspliced'])

AnnData object with n_obs × n_vars = 3186 × 2000
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'sample', 'S.Score', 'G2M.Score', 'Phase', 'integrated_snn_res.0.5', 'seurat_clusters', 'palantir_clusters', 'mono1', 'neu2', 'dc3', 'baso4', 'ery5', 'eos6', 'mep7', 'gmp8', 'cell_type', 'integrated_snn_res.0.4', 'integrated_snn_res.2', 'cell_type2', 'DF_score', 'DF_class', 'orig.lib', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'celltype', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'velocity_self_transition'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'highly_variable_genes', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes'
    uns: 'cell1_list', 'cell2_list_exp', 'cell_type2_co

In [3]:
SEED = 2024
torch.manual_seed(SEED)
np.random.seed(SEED)
vv.preprocess(adata, n_gene=2000)
vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0')

Filtered out 1105 genes that are detected 10 counts (shared).
Normalized count data: X, spliced, unspliced.
Skip filtering by dispersion since number of variables are less than `n_top_genes`.
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Keep raw unspliced/spliced count data.
Estimating ODE parameters...


100%|██████████| 895/895 [00:05<00:00, 152.00it/s]


Detected 468 velocity genes.
Estimating the variance...


100%|██████████| 895/895 [00:00<00:00, 5761.38it/s]

Initialization using the steady-state and dynamical models.


Reinitialize the regular ODE parameters based on estimated global latent time.


100%|██████████| 895/895 [00:00<00:00, 1485.01it/s]


3 clusters detected based on gene co-expression.
(0.45, 0.30969210995067337), (0.55, 0.6759994059560238)
(0.49, 0.21898765100307582), (0.51, 0.675265495558771)
(0.54, 0.6242421151380637), (0.46, 0.20123710624255123)
KS-test result: [0. 0. 0.]
Initial induction: 455, repression: 440/895


In [4]:
config = {
    # You can change any hyperparameters here!
    'learning_rate': 1e-3,
    'learning_rate_ode': 2e-3,
    'learning_rate_post': 1e-3
}
vae.train(adata,
          config=config,
          plot=False,
          #gene_plot=gene_plot,
          #figure_path=figure_path,
          embed='pca')

--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 18, test iteration: 34
*********                      Stage  2                       *********
*********             Velocity Refinement Round 1             *********


100%|██████████| 3186/3186 [00:01<00:00, 1699.99it/s]


Percentage of Invalid Sets: 0.026
Average Set Size: 64
Change in noise variance: 0.8246
*********             Velocity Refinement Round 2             *********
Change in noise variance: 0.0032
Change in x0: 0.0776
*********             Velocity Refinement Round 3             *********
*********     Round 3: Early Stop Triggered at epoch 2207.    *********
Change in noise variance: 0.0013
Change in x0: 0.0561
*********             Velocity Refinement Round 4             *********
Change in noise variance: 0.0004
Change in x0: 0.0692
*********             Velocity Refinement Round 5             *********
Stage 2: Early Stop Triggered at round 4.
*********              Finished. Total Time =   0 h : 52 m : 19 s             *********
Final: Train ELBO = 753.865,	Test ELBO = 594.980


In [5]:
adata

AnnData object with n_obs × n_vars = 3186 × 895
    obs: 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'sample', 'S.Score', 'G2M.Score', 'Phase', 'integrated_snn_res.0.5', 'seurat_clusters', 'palantir_clusters', 'mono1', 'neu2', 'dc3', 'baso4', 'ery5', 'eos6', 'mep7', 'gmp8', 'cell_type', 'integrated_snn_res.0.4', 'integrated_snn_res.2', 'cell_type2', 'DF_score', 'DF_class', 'orig.lib', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'nCount_ambiguous', 'nFeature_ambiguous', 'celltype', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'n_genes'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'highly_variable_genes', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes', 'init_mode', 'w_init'
    uns: 'cell1_list',

In [6]:
vae.save_model("./veloVAE", 'encoder_vae', 'decoder_vae')
vae.save_anndata(adata, 'vae', "./veloVAE", file_name="veloVAE.h5ad")